In [1]:
ROOT = 'https://www.saksfifthavenue.com/Men/Apparel/T-Shirts/shop/_/N-52kmqv/Ne-6lvnb5?FOLDER%3C%3Efolder_id=2534374306652711&Nao={}'

In [2]:
OFFSETS = [0, 150, 300, 450, 600]

In [3]:
ROOT.format(OFFSETS[-1])

'https://www.saksfifthavenue.com/Men/Apparel/T-Shirts/shop/_/N-52kmqv/Ne-6lvnb5?FOLDER%3C%3Efolder_id=2534374306652711&Nao=600'

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
# Headers 
agent = {"user-agent":
         "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"}

# Send request
url = ROOT.format(0)
print(url)
resp = requests.get(url, headers=agent)
soup = BeautifulSoup(resp.content, 'html.parser')

https://www.saksfifthavenue.com/Men/Apparel/T-Shirts/shop/_/N-52kmqv/Ne-6lvnb5?FOLDER%3C%3Efolder_id=2534374306652711&Nao=0


In [6]:
import re
from bs4 import BeautifulSoup, SoupStrainer

"""
In this step, build a list of search links
"""

SHIRT_LINKS = set()

for offset in OFFSETS:
    url = ROOT.format(offset)
    resp = requests.get(url, headers=agent)
    soup = BeautifulSoup(resp.content, 'html.parser')
    for link in soup.findAll('a', attrs={'href': re.compile("/product")}):
        l = link.get('href')
        if 'product' in l:
            SHIRT_LINKS.add(l)

SHIRT_LINKS = list(SHIRT_LINKS)

In [8]:
print(len(SHIRT_LINKS))

177


In [65]:
# Mess around with getting shirt images
import json

"""
Build function to generate all of the image links.
"""

def get_metadata(soup):
    metadata = soup.find('script', attrs={"type": "application/ld+json"})
    
    # Find the product code
    page_data = soup.find('script', attrs={"type": "text/javascript"})
    page_data = page_data.string.split("=")[1][:-2]
    
    # Find the final closing brace of the first complete json
    s = metadata.contents[0]
    left = s.index("{")
    ctr = 1
    index = left + 1
    right = None
    while index < len(s) and ctr != 0:
        if s[index] == "{":
            ctr += 1
        elif s[index] == "}":
            ctr -= 1
            right = index
        index += 1
    return s[left:right + 1], page_data
    
    
def get_image_links(shirt_link):
    # Curl the page and parse into bs4.
    shirt_link = f"https://www.saksfifthavenue.com/{shirt_link}"
    resp = requests.get(shirt_link, headers=agent)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    # Get the json metadata div from the soup.
    metadata, product_metadata = get_metadata(soup)
    metadata_js = json.loads(metadata)
    product_metadata_js = json.loads(product_metadata)
    
    # Navigate through the json for the shirt links in the media construction.
    images = metadata_js.get("image")
    product_id = product_metadata_js.get("products")[0].get("code")
    return images, product_id


"""
Let's give this a test drive.
"""
print(SHIRT_LINKS[0])
get_image_links(SHIRT_LINKS[0])


/product/stone-island-logo-patch-t-shirt-0400014407147.html?dwvar_0400014407147_color=BLACK


(['https://image.s5a.com/is/image/saks/0400014407147_BLACK?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0',
  'https://image.s5a.com/is/image/saks/0400014407147_BLACK_ASTL?wid=984&hei=1312&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0',
  'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A1?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0',
  'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A2?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0',
  'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A3?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0',
  'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A5?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0',
  'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A6?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0'],
 '0400014407147')

In [66]:
import os

os.listdir(".")
print(os.path.isdir("dataset"))

False


In [68]:
"""
Let's write a utility to download an image.
"""

import requests # to get image from the web
import shutil # to save it locally
import os

def write_image_to_disk(image_url, dst, root):    
    #Set up the image URL and filename
    filename = image_url.split("/")[-1]

    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
        
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(dst, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    else:
        print('Image Couldn\'t be retreived')
        
def write_image_set_to_disk(image_set, product_id):
    # Declare root for all images
    root = "/home/rajkinra23/git/drip_vision/data_pipeline/dataset/"
    
    # Create the directory for the image set, if it doesn't already exist. 
    image_set_root = os.path.join(root, product_id)
    if not os.path.isdir(image_set_root):
        os.mkdir(image_set_root)
        
        # Write images to the root
        for image_url in image_set:
            img = "{}.jpg".format(image_url.split('/')[-1])
            dst = os.path.join(image_set_root, img)
            write_image_to_disk(image_url, dst, image_set_root)
            
        # Return that we wrote images
        return True
    
    # Otherwise, we skip.
    else:
        print("Images already written to: {}".format(image_set_root))
        return False
            
# Test drive it
# write_image_set_to_disk(get_image_links(SHIRT_LINKS[0]))
links, pid = get_image_links(SHIRT_LINKS[0])
write_image_set_to_disk(links, pid)

Images already written to: /home/rajkinra23/git/drip_vision/data_pipeline/dataset/0400014407147


False

In [30]:
print(links)
write_image_set_to_disk(links)

['https://image.s5a.com/is/image/saks/0400014407147_BLACK?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0', 'https://image.s5a.com/is/image/saks/0400014407147_BLACK_ASTL?wid=984&hei=1312&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0', 'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A1?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0', 'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A2?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0', 'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A3?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0', 'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A5?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0', 'https://image.s5a.com/is/image/saks/0400014407147_BLACK_A6?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0']
/home/rajkinra23/git/drip_vision/data_pipeline/dataset/0400014407147_BLACK?wid=480&hei=640&qlt=90&resMode=sharp2&op_usm=0.9,1.0,8,0


False

In [144]:
"""
Execute the entire pipeline
"""

import time

def pipeline():
    for shirt_link in SHIRT_LINKS:
        image_links, product_id = get_image_links(shirt_link)
        if write_image_set_to_disk(image_links, product_id):
            time.sleep(2)
        print("Images downloaded for {}".format(shirt_link))
        
pipeline()


Images downloaded for https://www.saksfifthavenue.com/prps-graphic-t-shirt/product/0400012803207?FOLDER%3C%3Efolder_id=2534374306652711&R=190173652097&P_name=Prps&N=306652711&bmUID=nh.ahdl
Images downloaded for https://www.saksfifthavenue.com/valentino-vltn-collar-t-shirt/product/0400012206559?FOLDER%3C%3Efolder_id=2534374306652711&R=8054738167308&P_name=Valentino&N=306652711&bmUID=nh.Bnwm
Images downloaded for https://www.saksfifthavenue.com/hugo-dumans-graphic-t-shirt/product/0400012764369?FOLDER%3C%3Efolder_id=2534374306652711&R=754117967014&P_name=HUGO&N=306652711&bmUID=nh.Bnvw
Images downloaded for https://www.saksfifthavenue.com/sol-angeles-monterey-striped-t-shirt/product/0400012281196?FOLDER%3C%3Efolder_id=2534374306652711&R=840007963997&P_name=Sol+Angeles&N=306652711&bmUID=nh.d01f
Images downloaded for https://www.saksfifthavenue.com/helmut-lang-long-sleeve-pocket-t-shirt/product/0400012577248?FOLDER%3C%3Efolder_id=2534374306652711&R=883389766558&P_name=Helmut+Lang&N=306652711

Images downloaded for https://www.saksfifthavenue.com/heron-preston-embroidered-bird-graphic-t-shirt/product/0400012835652?FOLDER%3C%3Efolder_id=2534374306652711&R=8050848182583&P_name=Heron+Preston&N=306652711&bmUID=nh.aheI
Images downloaded for https://www.saksfifthavenue.com/burberry-jwear-logo-t-shirt/product/0400012342062?FOLDER%3C%3Efolder_id=2534374306652711&R=5045621954283&P_name=Burberry&N=306652711&bmUID=nh.d01h
Images downloaded for https://www.saksfifthavenue.com/prps-tie-dye-t-shirt/product/0400012744484?FOLDER%3C%3Efolder_id=2534374306652711&R=190173652998&P_name=Prps&N=306652711&bmUID=nh.Bnuv
Images downloaded for https://www.saksfifthavenue.com/g-star-raw-raw-boxed-logo-graphic-t-shirt/product/0400012395415?FOLDER%3C%3Efolder_id=2534374306652711&R=8719769099027&P_name=G-Star+RAW&N=306652711&bmUID=nh.d01j
Images downloaded for https://www.saksfifthavenue.com/sol-angeles-white-out-crewneck-t-shirt/product/0400012280807?FOLDER%3C%3Efolder_id=2534374306652711&R=840007963270

Images downloaded for https://www.saksfifthavenue.com/g-star-raw-3d-raw-graphic-t-shirt/product/0400012741545?FOLDER%3C%3Efolder_id=2534374306652711&R=8719769406733&P_name=G-Star+RAW&N=306652711&bmUID=nh.ahen
Images downloaded for https://www.saksfifthavenue.com/john-elliott-classic-crewneck-tee/product/0400089205589?FOLDER%3C%3Efolder_id=2534374306652711&R=841718101180&P_name=John+Elliott&N=306652711&bmUID=nh.Bnx4
Images downloaded for https://www.saksfifthavenue.com/stella-mccartney-unisex-stonewall-we-are-open-tee/product/0400011123263?FOLDER%3C%3Efolder_id=2534374306652711&R=840059741178&P_name=Stella+McCartney&N=306652711&bmUID=nh.cY2N
Images downloaded for https://www.saksfifthavenue.com/theory-regular-fit-marble-racer-t-shirt/product/0400012774780?FOLDER%3C%3Efolder_id=2534374306652711&R=192648397922&P_name=Theory&N=306652711&bmUID=nh.ahcW
Images downloaded for https://www.saksfifthavenue.com/comme-des-garcons-play-beatles-british-flag-tee/product/0400096435546?FOLDER%3C%3Efolde

Images downloaded for https://www.saksfifthavenue.com/orlebar-brown-ob-t-linen-t-shirt/product/0400097899658?FOLDER%3C%3Efolder_id=2534374306652711&R=5056128613238&P_name=Orlebar+Brown&N=306652711&bmUID=nh.Bnx0
Images downloaded for https://www.saksfifthavenue.com/giorgio-armani-ribbed-t-shirt/product/0400012009421?FOLDER%3C%3Efolder_id=2534374306652711&R=8058644570412&P_name=Giorgio+Armani&N=306652711&bmUID=nh.d026
Images downloaded for https://www.saksfifthavenue.com/givenchy-embroidered-logo-regular-fit-t-shirt/product/0400012766466?FOLDER%3C%3Efolder_id=2534374306652711&R=3665963008110&P_name=Givenchy&N=306652711&bmUID=nh.ahew
Images downloaded for https://www.saksfifthavenue.com/hugo-dneaker-graphic-t-shirt/product/0400012921047?FOLDER%3C%3Efolder_id=2534374306652711&R=791725042610&P_name=HUGO&N=306652711&bmUID=nh.ahd1
Images downloaded for https://www.saksfifthavenue.com/dsquared2-icons-twins-graphic-t-shirt/product/0400012583595?FOLDER%3C%3Efolder_id=2534374306652711&R=805618542

Images downloaded for https://www.saksfifthavenue.com/givenchy-degrade-signature-regular-fit-t-shirt/product/0400012766436?FOLDER%3C%3Efolder_id=2534374306652711&R=3665963014289&P_name=Givenchy&N=306652711&bmUID=nh.ahet
Images downloaded for https://www.saksfifthavenue.com/corneliani-solid-stretch-t-shirt/product/0400012068707?FOLDER%3C%3Efolder_id=2534374306652711&R=191180205320&P_name=Corneliani&N=306652711&bmUID=nh.d023
Images downloaded for https://www.saksfifthavenue.com/giorgio-armani-basic-crewneck-tee/product/0400098817418?FOLDER%3C%3Efolder_id=2534374306652711&R=8055353186989&P_name=Giorgio+Armani&N=306652711&bmUID=nh.Bnwr
Images downloaded for https://www.saksfifthavenue.com/balmain-neon-logo-print-t-shirt/product/0400012519684?FOLDER%3C%3Efolder_id=2534374306652711&R=3615881322966&P_name=Balmain&N=306652711&bmUID=nh.ahcl
Images downloaded for https://www.saksfifthavenue.com/balenciaga-soccer-t-shirt/product/0400013087615?FOLDER%3C%3Efolder_id=2534374306652711&R=889300604046&

...

Images downloaded for https://www.saksfifthavenue.com/brunello-cucinelli-silk-blend-t-shirt/product/0400012063621?FOLDER%3C%3Efolder_id=2534374306652711&R=8054732000458&P_name=Brunello+Cucinelli&N=306652711&bmUID=nh.d012
Images downloaded for https://www.saksfifthavenue.com/moncler-logo-long-sleeve-cotton-t-shirt/product/0400012921320?FOLDER%3C%3Efolder_id=2534374306652711&R=8053308114117&P_name=Moncler&N=306652711&bmUID=nh.ahej
Images downloaded for https://www.saksfifthavenue.com/mostly-heard-rarely-seen-peek-cotton-tee/product/0400012278491?FOLDER%3C%3Efolder_id=2534374306652711&R=841242185694&P_name=Mostly+Heard+Rarely+Seen&N=306652711&bmUID=nh.d01x
Images downloaded for https://www.saksfifthavenue.com/hugo-durned-logo-tape-t-shirt/product/0400012765046?FOLDER%3C%3Efolder_id=2534374306652711&R=754117795730&P_name=HUGO&N=306652711&bmUID=nh.BnvB
Images downloaded for https://www.saksfifthavenue.com/prps-poster-graphic-t-shirt/product/0400012396721?FOLDER%3C%3Efolder_id=25343743066527

Images downloaded for https://www.saksfifthavenue.com/balenciaga-drop-shoulder-t-shirt/product/0400011916151?FOLDER%3C%3Efolder_id=2534374306652711&R=889300506487&P_name=Balenciaga&N=306652711&bmUID=nh.d02j
Images downloaded for https://www.saksfifthavenue.com/marcelo-burlon-all-over-county-t-shirt/product/0400012047334?FOLDER%3C%3Efolder_id=2534374306652711&R=400120473362&P_name=Marcelo+Burlon&N=306652711&bmUID=nh.d00u
Images downloaded for https://www.saksfifthavenue.com/g-star-raw-dip-dye-logo-t-shirt/product/0400012467207?FOLDER%3C%3Efolder_id=2534374306652711&R=8719769105742&P_name=G-Star+RAW&N=306652711&bmUID=nh.d00C
Images downloaded for https://www.saksfifthavenue.com/maison-labiche-cotton-holidays-tee/product/0400010227235?FOLDER%3C%3Efolder_id=2534374306652711&R=3664021205072&P_name=Maison+Labiche&N=306652711&bmUID=nh.ahcN
Images downloaded for https://www.saksfifthavenue.com/amiri-falcon-logo-graphic-t-shirt/product/0400012797812?FOLDER%3C%3Efolder_id=2534374306652711&R=1918

Images downloaded for https://www.saksfifthavenue.com/balmain-striped-logo-graphic-tee/product/0400011839649?FOLDER%3C%3Efolder_id=2534374306652711&R=3615881002639&P_name=Balmain&N=306652711&bmUID=nh.d01p
Images downloaded for https://www.saksfifthavenue.com/saks-fifth-avenue-collection-cotton-t-shirt/product/0400098987128?FOLDER%3C%3Efolder_id=2534374306652711&R=400989871484&P_name=Saks+Fifth+Avenue&N=306652711&bmUID=nh.cY2m
Images downloaded for https://www.saksfifthavenue.com/givenchy-logo-t-shirt/product/0400099791927?FOLDER%3C%3Efolder_id=2534374306652711&R=3615205346029&P_name=Givenchy&N=306652711&bmUID=nh.cY2g
Images downloaded for https://www.saksfifthavenue.com/helmut-lang-standard-brand-cotton-t-shirt/product/0400012567282?FOLDER%3C%3Efolder_id=2534374306652711&R=883389770777&P_name=Helmut+Lang&N=306652711&bmUID=nh.BnwJ
Images downloaded for https://www.saksfifthavenue.com/john-elliott-anti-expo-t-shirt/product/0400012700756?FOLDER%3C%3Efolder_id=2534374306652711&R=1932360895

Images downloaded for https://www.saksfifthavenue.com/prps-poster-graphic-t-shirt/product/0400012396722?FOLDER%3C%3Efolder_id=2534374306652711&R=190173614682&P_name=Prps&N=306652711&bmUID=nh.Bnxb
Images downloaded for https://www.saksfifthavenue.com/burberry-justin-logo-tee/product/0400011858340?FOLDER%3C%3Efolder_id=2534374306652711&R=5045621142109&P_name=Burberry&N=306652711&bmUID=nh.cY0K
Images downloaded for https://www.saksfifthavenue.com/sunspel-classic-crewneck-cotton-tee/product/0400095971116?FOLDER%3C%3Efolder_id=2534374306652711&R=5055611500048&P_name=Sunspel&N=306652711&bmUID=nh.cY1F
Images downloaded for https://www.saksfifthavenue.com/vilebrequin-st-tropez-t-shirt/product/0400010748506?FOLDER%3C%3Efolder_id=2534374306652711&R=7613409145635&P_name=Vilebrequin&N=306652711&bmUID=nh.ahek
Images downloaded for https://www.saksfifthavenue.com/comme-des-garcons-play-bottom-heart-tee/product/0400099215887?FOLDER%3C%3Efolder_id=2534374306652711&R=400992158909&P_name=Comme+des+Garco

Images downloaded for https://www.saksfifthavenue.com/rag-bone-classic-long-sleeve-henley/product/0400097654421?FOLDER%3C%3Efolder_id=2534374306652711&R=190297358059&P_name=Rag+%26+Bone&N=306652711&bmUID=nh.cY14
Images downloaded for https://www.saksfifthavenue.com/eleven-paris-lisa-graffiti-graphic-t-shirt/product/0400011410724?FOLDER%3C%3Efolder_id=2534374306652711&R=829468699676&P_name=Eleven+Paris&N=306652711&bmUID=nh.cY1I
Images downloaded for https://www.saksfifthavenue.com/john-varvatos-hand-rose-cotton-t-shirt/product/0400012434360?FOLDER%3C%3Efolder_id=2534374306652711&R=190802781532&P_name=John+Varvatos&N=306652711&bmUID=nh.d00N
Images downloaded for https://www.saksfifthavenue.com/g-star-raw-box-logo-graphic-tee/product/0400012467170?FOLDER%3C%3Efolder_id=2534374306652711&R=8719769099171&P_name=G-Star+RAW&N=306652711&bmUID=nh.d00A
Images downloaded for https://www.saksfifthavenue.com/polo-ralph-lauren-jersey-graphic-logo-t-shirt/product/0400012662492?FOLDER%3C%3Efolder_id=25